Please install nnUNet v1, and run this notebook using the nnUNet conda/virtual env. 

### Firstly setting up environment variables

Add your path here instead and copy paste in terminal in your nnUNet Environment:
```
export nnUNet_raw_data_base="/location/to/nnUNet_raw_data_base"
export nnUNet_preprocessed="/location/to/nnUNet_preprocessed"
export RESULTS_FOLDER="/location/to/nnUNet_trained_models"
```

### Step 1: Transferring of Data

In [1]:
import os 
from natsort import natsorted
from glob2 import glob

In [2]:
def nnUnet_transfer_images(patients_dir, list_of_modalities, output_dir, exp_name):
    all_patients_dir = natsorted(glob(patients_dir + '/*'))
    
    for i, p in enumerate(all_patients_dir):
        for j, modality in enumerate(list_of_modalities):
            src_file = p + '/' + p.split('/')[-1] + '_' + modality + '.nii.gz'
            dst_file = output_dir + '/' + exp_name + '_' + '{:04}'.format(i+1) + '_000' + str(j) + '.nii.gz' 

            #print(dst_file)
            os.system('cp -r '+ src_file + ' ' + dst_file)    
            
def nnUnet_transfer_segs(patients_dir, output_dir, exp_name):
    
    all_patients_dir = natsorted(glob(patients_dir + '/*'))
    
    for i, p in enumerate(all_patients_dir):
        src_file = p + '/' + p.split('/')[-1] + '_seg.nii.gz'
        dst_file = output_dir + '/' + exp_name + '_'  + '{:04}'.format(i+1) + '.nii.gz'

        #print(dst_file)
        os.system('cp -r '+ src_file + ' ' + dst_file)  

#### Creating Files directory

Change the experiment name and path for your project/experiment

In [3]:
os.system('mkdir /location/to/TaskXXX_UCSF_BMSR')
os.system('mkdir /location/to/TaskXXX_UCSF_BMSR/imagesTr')
os.system('mkdir /location/to/TaskXXX_UCSF_BMSR/imagesTs')
os.system('mkdir /location/to/TaskXXX_UCSF_BMSR/labelsTr')
os.system('mkdir /location/to/TaskXXX_UCSF_BMSR/labelsTs')

0

#### Transferring Images

Change the experiment name and path for your project/experiment

In [4]:
nnUnet_transfer_images(patients_dir = '/location/to/patients_1/', 
                     list_of_modalities = ['flair','t1','t2'], 
                     output_dir = '/location/to/TaskXXX_UCSF_BMSR/imagesTr', 
                     exp_name = 'TrialExp')

In [5]:
nnUnet_transfer_images(patients_dir = '/location/to/patients_2/', 
                     list_of_modalities = ['flair','t1','t2'], 
                     output_dir = '/location/to/TaskXXX_UCSF_BMSR/imagesTs', 
                     exp_name = 'TrialExp')

#### Transferring Segs

Change the experiment name and path for your project/experiment

In [6]:
nnUnet_transfer_segs(patients_dir = '/location/to/patients_1/', 
                     output_dir = '/location/to/TaskXXX_UCSF_BMSR/labelsTr', 
                     exp_name = 'TrialExp')

In [7]:
nnUnet_transfer_segs(patients_dir = '/location/to/patients_2/', 
                     output_dir = '/location/to/TaskXXX_UCSF_BMSR/labelsTs', 
                     exp_name = 'TrialExp')

#### Creating dataset json

In [8]:
from typing import Tuple
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *

In [9]:
def get_identifiers_from_splitted_files(folder: str):
    uniques = np.unique([i[:-12] for i in subfiles(folder, suffix='.nii.gz', join=False)])
    return uniques


def generate_dataset_json(output_file: str, imagesTr_dir: str, imagesTs_dir: str, modalities: Tuple,
                          labels: dict, dataset_name: str, license: str = "hands off!", dataset_description: str = "",
                          dataset_reference="", dataset_release='0.0'):
    """
    :param output_file: This needs to be the full path to the dataset.json you intend to write, so
    output_file='DATASET_PATH/dataset.json' where the folder DATASET_PATH points to is the one with the
    imagesTr and labelsTr subfolders
    :param imagesTr_dir: path to the imagesTr folder of that dataset
    :param imagesTs_dir: path to the imagesTs folder of that dataset. Can be None
    :param modalities: tuple of strings with modality names. must be in the same order as the images (first entry
    corresponds to _0000.nii.gz, etc). Example: ('T1', 'T2', 'FLAIR').
    :param labels: dict with int->str (key->value) mapping the label IDs to label names. Note that 0 is always
    supposed to be background! Example: {0: 'background', 1: 'edema', 2: 'enhancing tumor'}
    :param dataset_name: The name of the dataset. Can be anything you want
    :param license:
    :param dataset_description:
    :param dataset_reference: website of the dataset, if available
    :param dataset_release:
    :return:
    """
    train_identifiers = get_identifiers_from_splitted_files(imagesTr_dir)

    if imagesTs_dir is not None:
        test_identifiers = get_identifiers_from_splitted_files(imagesTs_dir)
    else:
        test_identifiers = []

    json_dict = {}
    json_dict['name'] = dataset_name
    json_dict['description'] = dataset_description
    json_dict['tensorImageSize'] = "4D"
    json_dict['reference'] = dataset_reference
    json_dict['licence'] = license
    json_dict['release'] = dataset_release
    json_dict['modality'] = {str(i): modalities[i] for i in range(len(modalities))}
    json_dict['labels'] = {str(i): labels[i] for i in labels.keys()}

    json_dict['numTraining'] = len(train_identifiers)
    json_dict['numTest'] = len(test_identifiers)
    json_dict['training'] = [
        {'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i
        in
        train_identifiers]
    json_dict['test'] = ["./imagesTs/%s.nii.gz" % i for i in test_identifiers]

    if not output_file.endswith("dataset.json"):
        print("WARNING: output file name is not dataset.json! This may be intentional or not. You decide. "
              "Proceeding anyways...")
    save_json(json_dict, os.path.join(output_file))

In [10]:
generate_dataset_json(
    output_file = "/location/to/TaskXXX_UCSF_BMSR/dataset.json",
    imagesTr_dir = "/location/to/TaskXXX_UCSF_BMSR/imagesTr",
    imagesTs_dir = "/location/to/TaskXXX_UCSF_BMSR/imagesTs",
    modalities = ('flair', 't1', 't2'),
    labels = {
        "0": "background",
        "1": "FLAIR abnormal singal"
    },
    dataset_name = "Trialexp",
    license = "hands off!",
    dataset_description = "nnUnet experiment for Trialexp",
    dataset_reference = "",
    dataset_release = "1.0"
)

### Step 2: Run Preprocessing


Copy and run this on the terminal.

```
nnUNet_plan_and_preprocess -t XXX --verify_dataset_integrity
```

If everything is okay, then it will automatically run preprocessing. If not, there could be errors in your image and seg size then you need to fix it. OR if there is some small orientation differences also, it won't run automatically, here you can still do preprocessing by running the following

```
nnUNet_plan_and_preprocess -t XXX 
```

### Step 3: Run training

To run the training run the following on the terminal

```
CUDA_VISIBLE_DEVICES=3 nnUNet_train 3d_fullres nnUNetTrainerV2 TaskXXX_MYTASK FOLD --npz
```

FOLD = 0,1,2,3,4,all

To stop training anytime, just press CTRL-C

To start again
```
nnUNet_train 3d_fullres nnUNetTrainerV2 TaskXXX_MYTASK FOLD --npz -c
```

### Step 4: Run Inference

To run the inference run the following on the terminal

```
CUDA_VISIBLE_DEVICES=3 nnUNet_predict -i /location/to/TaskXXX_UCSF_BMSR/imagesTs -o /location/to/TaskXXX_UCSF_BMSR/imagesTs_pred -t XXX -m 3d_fullres -f all --save_npz 
```
